# Text Analysis With Python 

## Introduction - Lorin

## Python Packages often used for Text Analysis - Rolando
- Directly related to Text Analysis
    - NLTK
    - spaCy
    - Textblob
    - Gensim
    - Transformers

- Useful tools for text analysis
    - Pandas
    - Scikit-Learn
    - Matplotlib


## Mini-project 1: Word Frequencies (1:05) - Rolando

#### Data: [Jane Eyre - Charlotte Brontë](https://www.gutenberg.org/files/1260/1260-h/1260-h.htm)
#### Tools: NLTK, Scitkit-learn, Pandas
#### Method: Simple N-grams (Document vs Chapters), Maybe TF-IDF, 

## Mini-project 2: Classification (1:15) - Lorin

#### Data: [On the Books Laws](https://cdr.lib.unc.edu/concern/data_sets/v405sk89q?locale=en)
#### Tools: Scitkit-learn, Pandas, 
#### Methods: Supervised (Jim Crow vs. Non-Jim Crow) vs Unsupervised (Topic-modeling)
#### Helpful: [Comparing classifiers](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html)


## Mini-project 3: Sentiment Analysis (1:35) - Rolando

#### Data: [Chaptgpt Sentiment Analysis](https://www.kaggle.com/datasets/charunisa/chatgpt-sentiment-analysis)
#### Tools: Textblob, NLTK, spaCy, Transformers
#### Methods: Dictionary (+ rule-based) vs. Transformer-based approach